# zuco

In [ ]:
import scipy.io as io
# read data/zuco/task1/Matlab_files/resultsZAB_SR.mat
sentence_data = io.loadmat('data/zuco/task1/Matlab_files/resultsZAB_SR.mat', squeeze_me=True, struct_as_record=False,)['sentenceData']

In [6]:
import numpy as np
import pandas as pd
sn_data = sentence_data[0]
dur = np.array(sn_data.allFixations.duration)
x = np.array(sn_data.allFixations.x)
y = np.array(sn_data.allFixations.y)


In [12]:
sn_data.content

'Presents a good case while failing to provide a reason for us to care beyond the very basic dictums of human decency.'

In [ ]:
word_boundary = sn_data.wordbounds
df = pd.DataFrame([], columns=['sn', 'wn', 'fl', 'dur'])
for fix_idx in range(x.shape[0]):
    word_idx = np.where(
        (word_boundary[:, 0] <= x[fix_idx]) &
        (word_boundary[:, 2] >= x[fix_idx]) &
        (word_boundary[:, 1] <= y[fix_idx]) &
        (word_boundary[:, 3] >= y[fix_idx]),
    )
    assert len(word_idx) == 1, 'more than 1 word is matched!'
    # skip fixations outside all boundary boxes
    if word_idx[0].size == 0:
        continue
    fl = ((x[fix_idx] - word_boundary[word_idx, 0]) /
            (word_boundary[word_idx, 2] - word_boundary[word_idx, 0]))[0][0]
    df_tmp = pd.DataFrame(
        [[
            1,  word_idx[0][0] + 1,
            fl, dur[fix_idx],
        ]], columns=['sn', 'wn', 'fl', 'dur'],
    )
    df = pd.concat([df, df_tmp])


In [11]:
words = pd.read_csv("data/zuco/task1/Matlab_files/scanpath.csv", sep='\t')
# check is there is nan in words
words.isnull().values.any()

# check if there is nan or '' in column 'WORD'
words[words['CURRENT_FIX_INTEREST_AREA_LABEL'].isnull().values]
words[words['CURRENT_FIX_INTEREST_AREA_LABEL'] == '']

,id,sn,nw,wn,fl,dur,CURRENT_FIX_INTEREST_AREA_LABEL


In [5]:
import pandas as pd
path = "data/zuco/task2/Matlab_files/scanpath.csv"
df = pd.read_csv(path, sep='\t')
df['id'].unique()


array(['ZAB', 'ZDM', 'ZDN', 'ZGW', 'ZJM', 'ZJN', 'ZJS', 'ZKB', 'ZKH',
       'ZKW', 'ZMG', 'ZPH'], dtype=object)

In [30]:
import pandas as pd
import numpy as np
import ast

df = pd.read_csv("data/zuco/task2/fixation.csv", sep=',')
sentences = df['sn'].unique()
df_avg = pd.DataFrame([], columns=['sn', 'list_dur'])

for sn in sentences:
    df_tmp = df[df['sn'] == sn]
    # Convert string representations to actual lists
    df_tmp['list_dur'] = df_tmp['list_dur'].apply(ast.literal_eval)
    
    # Calculate the average fixation duration
    avg_dur = np.nanmean(df_tmp['list_dur'].values.tolist(), axis=0)

    df_avg = pd.concat([df_avg, pd.DataFrame({'sn': [sn], 'list_dur': [avg_dur.tolist()]})], ignore_index=True)

df_avg.to_csv("data/zuco/task2/fixation_avg.csv", index=False)

    


/var/folders/ng/jh0twkn55sbds4qg4kkndgq80000gn/T/ipykernel_6601/2755516044.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['list_dur'] = df_tmp['list_dur'].apply(ast.literal_eval)
/var/folders/ng/jh0twkn55sbds4qg4kkndgq80000gn/T/ipykernel_6601/2755516044.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['list_dur'] = df_tmp['list_dur'].apply(ast.literal_eval)
/var/folders/ng/jh0twkn55sbds4qg4kkndgq80000gn/T/ipykernel_6601/2755516044.py:12: SettingWithCopyWarning: 
A value is trying to 

In [ ]:
a = [1, 2, 3]
b = [4, 5, 6]

df = pd.DataFrame(columns=['list'])
df['list'] = [a]
df['list'].append({'list': b})
df



In [22]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
text = "Ford was born on a prosperous farm in Springwells Township (now in the city of Dearborn, Michigan) owned by his parents, William Ford (1826-1905) and Mary Litogot (c1839-1876), immigrants from County Cork, Ireland." 
text = text.strip() + "[MASK]" * 10
input_tokens = tokenizer(text)['input_ids']
attention_ids = tokenizer(text)['attention_mask']
input_tokens = torch.tensor(input_tokens).unsqueeze(0)
attention_ids = torch.tensor(attention_ids).unsqueeze(0)
torch.enable_grad()
model.eval()

A = model(input_tokens, attention_mask=attention_ids)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
import numpy as np
import string

def merge_bert_tokens(tokens, text, gradients, special_tokens=["[CLS]", "[SEP]", "[MASK]"]):
    gradients_list = []
    word_list = []
    word = ""
    word_gradients = 0
    # Merge tokens into the original words
    for i, token in enumerate(tokens):
        if token in special_tokens:
            continue
        if token.startswith("##"):
            word += token[2:]
            word_gradients += gradients[i]
        else:
            if word != "":
                word_list.append(word)
                gradients_list.append(word_gradients)
                word = ""
                word_gradients = 0
            word = token
            word_gradients = gradients[i]
    if word != "":
        word_list.append(word)
        gradients_list.append(word_gradients)
    # Merge further the words into the original text

    merged_gradients, merged_word_list = merge_further(word_list, text, gradients_list)

    return np.array(merged_gradients).squeeze(), merged_word_list


               

# Example usage:
bert_tokens = tokenizer.convert_ids_to_tokens(input_tokens[0])
# generate a randon gradient list with the same length of bert_tokens
random_gradients = np.random.rand(len(bert_tokens))
print(bert_tokens)

merged_gradients_bert, word_list = merge_bert_tokens(bert_tokens, text, random_gradients)
print(merged_gradients_bert)
print(word_list)


['[CLS]', 'ford', 'was', 'born', 'on', 'a', 'prosperous', 'farm', 'in', 'spring', '##well', '##s', 'township', '(', 'now', 'in', 'the', 'city', 'of', 'dear', '##born', ',', 'michigan', ')', 'owned', 'by', 'his', 'parents', ',', 'william', 'ford', '(', '1826', '-', '1905', ')', 'and', 'mary', 'lit', '##ogo', '##t', '(', 'c1', '##8', '##39', '-', '1876', ')', ',', 'immigrants', 'from', 'county', 'cork', ',', 'ireland', '.', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[MASK]', '[SEP]']
['ford', 'was', 'born', 'on', 'a', 'prosperous', 'farm', 'in', 'springwells', 'township', '(now', 'in', 'the', 'city', 'of', 'dearborn,', 'michigan)', 'owned', 'by', 'his', 'parents,', 'william', 'ford', '(1826-1905)', 'and', 'mary', 'litogot', '(c1839-1876),', 'immigrants', 'from', 'county', 'cork,', 'ireland.']
target:  ford
word:  ford
target:  was
word:  was
target:  born
word:  born
target:  on
word:  on
target:  a
word:  a
target:  prosperous
word:  prosp

In [ ]:
def merge_further(word_list, text, gradients):
    original_wrods = text.lower().split()
    print(original_wrods)
    merged_word_list = []
    merged_gradients = []
    tmp = ""
    word_gradients = 0
    word_count = 0
    
    for target in original_wrods:     
        
        for i, word in enumerate(word_list[word_count:]):
            
            if word == target:
                merged_word_list.append(word)
                merged_gradients.append(gradients[i])
                word_count += 1
                break
            elif word in target:
                tmp += word
                word_gradients += gradients[i]
                word_count += 1
            else:
                merged_word_list.append(tmp)
                merged_gradients.append(word_gradients)
                tmp = ""
                word_gradients = 0
                break
    if tmp != "":
        merged_word_list.append(tmp)
        merged_gradients.append(word_gradients)
                
    return merged_gradients, merged_word_list

text = "Ford was born on a prosperous farm in Springwells Township (now in the city of Dearborn, Michigan) owned by his parents, William Ford (1826-1905) and Mary Litogot (c1839-1876), immigrants from County Cork, Ireland."
merged_gradients, merged_word_list = merge_further(word_list, text, merged_gradients_bert)
print(merged_gradients)
print(merged_word_list)
print(len(merged_gradients))

In [10]:
import pandas as pd

df1 = pd.read_csv("data/zuco/task2/bert_sa.csv", sep=',')
df2 = pd.read_csv("data/zuco/task2/bert_saliency.csv", sep=',')
# check how many row are the same in l1_grad and l2_grad
for i in range(10):
    print("l1_grad: ", df1['l1_grad'][i])
    print("l2_grad: ", df2['l1_grad'][i])




l1_grad:  [0.03192919333795058, 0.02898076429554445, 0.041428676997888915, 0.0251700324024263, 0.05656898138821232, 0.09397514245328803, 0.039564720052740046, 0.06351361032077209, 0.03204903433507129, 0.04464570037890856, 0.11833061982565392, 0.0546785251846204, 0.12123815599891799, 0.11473017049654163, 0.13319667253146347]
l2_grad:  [0.05642387840067943, 0.05642387840067943, 0.05642387840067943, 0.05642387840067943, 0.05642387840067943, 0.1076378197306156, 0.05642387840067943, 0.05642387840067943, 0.05642387840067943, 0.05642387840067943, 0.1076378197306156, 0.05642387840067943, 0.05642387840067943, 0.05642387840067943, 0.1076378197306156]
l1_grad:  [0.009346331989774312, 0.008859001731342103, 0.024948736494661925, 0.01213385722536939, 0.026902473511407864, 0.024995006725556622, 0.013572374367183514, 0.04735268653291626, 0.008924590363080622, 0.022339351038069024, 0.010743657433649014, 0.009714729578523462, 0.026915029175828737, 0.02564648620906078, 0.06536152578318764, 0.090230914870

In [5]:
text = "djdk skdls sldksl"
text.strip() + "[MASK]" * 10

'djdk skdls sldksl[MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK][MASK]'

# sst

In [143]:
from datasets import load_dataset, concatenate_datasets

dataset = load_dataset("SetFit/sst5")
dataset = concatenate_datasets([dataset['train'], dataset['validation'], dataset['test']])

/Users/first_principle/opt/anaconda3/envs/scan/lib/python3.9/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [203]:
# convert the dataset to pandas dataframe
sst_df = pd.read_csv("data/sst/sst5.csv", sep=',')
# add a index column
sst_df['index'] = sst_df.index

In [204]:
import pandas as pd
import string
import re

label_df = pd.DataFrame([], columns=['sid', 'label'])
df = pd.read_csv("data/sst/test.csv", sep='\t')
# lower case all the sentences
df['CONTENT'] = df['CONTENT'].str.lower()
# remove the whitespaces in sentences for both df['CONTENT'] and sst_df['text']
df['CONTENT'] = df['CONTENT'].apply(lambda x: re.sub(r"\s+", "", x, flags=re.UNICODE))
sst_df['text'] = sst_df['text'].apply(lambda x: re.sub(r"\s+", "", x, flags=re.UNICODE))
# remove the punctuation in sentences for both df['CONTENT'] and sst_df['text']
df['CONTENT'] = df['CONTENT'].str.replace('[{}]'.format(string.punctuation), '')
sst_df['text'] = sst_df['text'].str.replace('[{}]'.format(string.punctuation), '')

overlap_list = []
# check the overlap between the sentences in df and dataset
for i in range(len(df)):
    if df['CONTENT'][i] in sst_df['text'].values:
        label_df = pd.concat([label_df, pd.DataFrame({'sid': [df['SN'][i]], 'label': [sst_df[sst_df['text'] == df['CONTENT'][i]]['label'].values[0]]})], ignore_index=True)
        overlap_list.append(sst_df[sst_df['text'] == df['CONTENT'][i]]['index'].values[0])
    else:
        print("sentence {} is not in the dataset".format(df['SN'][i]))
        label_df = pd.concat([label_df, pd.DataFrame({'sid': [df['SN'][i]], 'label': [-1]})], ignore_index=True)


/var/folders/ng/jh0twkn55sbds4qg4kkndgq80000gn/T/ipykernel_1940/3032391505.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['CONTENT'] = df['CONTENT'].str.replace('[{}]'.format(string.punctuation), '')
/var/folders/ng/jh0twkn55sbds4qg4kkndgq80000gn/T/ipykernel_1940/3032391505.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  sst_df['text'] = sst_df['text'].str.replace('[{}]'.format(string.punctuation), '')


In [199]:
# save the label_df to csv file
label_df.to_csv("data/sst/label.csv", index=False)

In [220]:
# count unique values in a column
label_df = pd.read_csv("data/sst/label.csv", sep=',')
label_df['label'].value_counts()

# replace label 1 with 0, 3 with 4
# label_df['label'] = label_df['label'].replace(1, 0)
# label_df['label'] = label_df['label'].replace(3, 4)
# label_df['label'].value_counts()


4    142
2    135
0    123
Name: label, dtype: int64

In [221]:
test_df = pd.read_csv("data/sst/sentence_content.csv", sep='\t')
# change the column name from 'CONTENT' to 'text', "SN" to sid
test_df = test_df.rename(columns={'CONTENT': 'text', 'SN': 'sid'})
# merge the test_df and label_df
test_df = pd.merge(test_df, label_df, on='sid')
# save the test_df to csv file
test_df.to_csv("data/sst/test.csv", index=False)

In [219]:
# save the label_df to csv file
label_df.to_csv("data/sst/label.csv", index=False)

In [213]:
sst_df = pd.read_csv("data/sst/sst5.csv", sep=',')
sst_df['index'] = sst_df.index
# remove the overlap sentences
sst_df = sst_df.drop(overlap_list)

# only keep the sentences with label 0, 2, 4
sst_df = sst_df[sst_df['label'] != 1]
sst_df = sst_df[sst_df['label'] != 3]
sst_df['label'].value_counts()

# train, validation split
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(sst_df, test_size=0.2, random_state=42)
train_df['label'].value_counts()
val_df['label'].value_counts()

2    417
4    356
0    270
Name: label, dtype: int64

In [222]:
# remove the index column and label_text column
train_df = train_df.drop(['index', 'label_text'], axis=1)
val_df = val_df.drop(['index', 'label_text'], axis=1)

In [223]:
# save the train_df and val_df to csv file
train_df.to_csv("data/sst/train.csv", index=False)
val_df.to_csv("data/sst/val.csv", index=False)

# wiki

In [26]:
import pickle
import pandas as pd

# Specify the path to your .p file
file_path = 'data/wiki/wiki_rel_zuco_filtered.p'

# Open the file in binary mode
with open(file_path, 'rb') as file:
    # Load the object from the file
    data = pickle.load(file)

# Now 'data' contains the object stored in the .p file

In [66]:
zuco_df = pd.read_csv("data/wiki/test.csv", sep='\t')

In [27]:
data.keys()

dict_keys(['train', 'test', 'wiki_in_zuco_samples', 'remaining_zuco'])

In [28]:
data_list = data['test']
test_df = pd.DataFrame(data_list, columns=['text', 'relation'])
data_list = data['train']
train_df = pd.DataFrame(data_list, columns=['text', 'relation'])
df = pd.concat([train_df, test_df], ignore_index=True)

In [29]:
df['relation'] = df['relation'].apply(lambda x: x[2] if len(x) >= 3 else None)

In [86]:
df[df['text'].str.contains("Henry Ford Company")]

,text,relation
144,"With his interest in race cars, he formed a se...",FOUNDER


In [37]:
# keep the rows with relation award, education, employer, founder, job_title, nationality, political_affiliation, visited and wife
desired_relations = ['award', 'education', 'employer', 'founder', 'job_title', 'nationality', 'political_affiliation', 'visited', 'wife']

# lower case the relation in df
df['relation'] = df['relation'].str.lower()
# Filter the DataFrame to keep only rows with the desired relation types
filtered_df = df[df['relation'].isin(desired_relations)]

filtered_df['relation'].value_counts()


visited                  304
job_title                249
education                123
nationality               86
employer                  65
award                     37
wife                      19
founder                    3
political_affiliation      3
Name: relation, dtype: int64

In [38]:
from sklearn.model_selection import train_test_split
# rename the column name from 'relation' to 'label'
filtered_df = filtered_df.rename(columns={'relation': 'label'})

# Assuming 'label' is the column containing your labels in the DataFrame
train_df, val_df = train_test_split(filtered_df, test_size=0.2, random_state=42, stratify=filtered_df['label'])

train_df['label'].value_counts()



visited                  243
job_title                199
education                 98
nationality               69
employer                  52
award                     30
wife                      15
political_affiliation      3
founder                    2
Name: label, dtype: int64

In [39]:

train_df.to_csv("data/wiki/train.csv", index=False)
val_df.to_csv("data/wiki/val.csv", index=False)

In [17]:
df = pd.read_csv("data/wiki/relations_task_specific.csv", delimiter=';', encoding='latin-1')
# drop first two columns
df = df.drop(df.columns[[0, 1]], axis=1)
# name the columns "text" and "relation"
df = df.rename(columns={'sentence': 'text', 'relation_type': 'relation'})

In [16]:
len(df)

407

In [18]:
test_df = pd.read_csv("data/wiki/test.csv", sep='\t')
# change the column name from 'CONTENT' to 'text', "SN" to sid
test_df = test_df.rename(columns={'CONTENT': 'text', 'SN': 'sid'})
len(test_df)

407

In [20]:
label_df = pd.DataFrame([], columns=['sid', 'label'])
for i in range(len(test_df)):
    if test_df['text'][i] in df['text'].values:
        label_df = pd.concat([label_df, pd.DataFrame({'sid': [test_df['sid'][i]], 'label': [df[df['text'] == test_df['text'][i]]['relation'].values[0]]})], ignore_index=True)
    else:
        label_df = pd.concat([label_df, pd.DataFrame({'sid': [test_df['sid'][i]], 'label': [-1]})], ignore_index=True)
        print("sentence {} is not in the dataset".format(test_df['sid'][i]))

sentence 349 is not in the dataset


In [21]:
#merge the test_df and label_df
test_df = pd.merge(test_df, label_df, on='sid')
# save the test_df to csv file
test_df.to_csv("data/wiki/test_label.csv", index=False)

In [36]:
test_df = pd.read_csv("data/wiki/test_filtered.csv", sep=',')
test_df['label'].value_counts()



job_title                49
award                    41
education                39
employer                 39
wife                     37
visited                  35
founder                  34
political_affiliation    34
nationality              27
Name: label, dtype: int64

# BNC

In [19]:
import pandas as pd
import numpy as np
with open('data/bnc.txt', 'r', encoding='latin-1') as file:
    # Read the entire contents of the file into a string
    content = file.read()

lines = [line.strip().split('\t') for line in content.split('\n') if line.strip()]

# Assuming the first row contains the column headers
headers = lines[0]

# Create a DataFrame using the remaining rows and the extracted headers
bnc_df = pd.DataFrame(lines[1:], columns=headers)

# Display the DataFrame
print(bnc_df)
# fill nan with 0 in column 'Freq'
bnc_df['Freq'] = bnc_df['Freq'].fillna(0)


            Word   PoS   Freq
0            the   Det  61847
1             of  Prep  29391
2            and  Conj  26817
3              a   Det  21626
4             in  Prep  18214
...          ...   ...    ...
7721           S   NoP     10
7722        pine   NoC     10
7723    borrowed  Verb     10
7724         bow   NoC     10
7725  disturbing   Adj     10

[7726 rows x 3 columns]


In [80]:
"uplifter" in bnc_df['Word'].values

False

In [27]:
import string
sst_df = pd.read_csv("data/sst/test.csv", sep=',')
freq_df = pd.DataFrame([], columns=['sid', 'freq'])
# lower case all the sentences
sst_df['text'] = sst_df['text'].str.lower()

for i in range(len(sst_df)):
    # for each word in the sentence, assign the frequency of the word to the sentence
    freq_list = []
    for word in sst_df['text'][i].split():
        # remove the punctuation in the word
        word = word.translate(str.maketrans('', '', string.punctuation))
        if word in bnc_df['Word'].values:
            freq = int(bnc_df[bnc_df['Word'] == word]['Freq'].values[0])
            if freq == 0:
                freq = 10
            inverse_freq = -np.log(freq/1000000)
            freq_list.append(inverse_freq)
        else:
            freq = 10
            inverse_freq = -np.log(freq/1000000)
            freq_list.append(inverse_freq)
        #round the frequency to 2 decimal places
        # are min- max-normalized
    freq_df = pd.concat([freq_df, pd.DataFrame({'sid': [sst_df['sid'][i]], 'freq': [freq_list]})], ignore_index=True)

# for each frequency list, do l1 normalization
freq_df['freq'] = freq_df['freq'].apply(lambda x: x/np.sum(x))

,sid,freq
0,1,"[0.0686353190789759, 0.023828669496768557, 0.0..."
1,2,"[0.058236233127732386, 0.059173316514611884, 0..."
2,3,"[0.08205439478903212, 0.05228818391871549, 0.0..."
3,4,"[0.21125659198476024, 0.2227070163000617, 0.07..."
4,5,"[0.09264356972064981, 0.09149785857613826, 0.1..."
...,...,...
395,396,"[0.05943338192979294, 0.03550343022227235, 0.1..."
396,397,"[0.06341240706009747, 0.025374886034659573, 0...."
397,398,"[0.0743598038481796, 0.06228827427235792, 0.04..."
398,399,"[0.0534265529132374, 0.027254889429436782, 0.0..."


In [22]:
import ast
def average_fixation(path):
    # average the fixation data of all the subjects
    df = pd.read_csv(path, sep=',')
    df_avg = pd.DataFrame(columns=['sid', 'list_dur'])
    sentences = df['sn'].unique()
    for sentence in sentences:
        df_sentence = df[df['sn'] == sentence]
        df_tmp = df_sentence['list_dur'].apply(ast.literal_eval)
        # normalize the fixation duration
        df_tmp = df_tmp.apply(lambda x: [i/sum(x) for i in x])
        avg_dur = np.mean(df_tmp.values.tolist(), axis=0)
        df_avg = pd.concat([df_avg, pd.DataFrame({'sid': sentence, 'list_dur': [avg_dur.tolist()]})], ignore_index=True)
    return df_avg

fix_df = average_fixation("data/zuco/task1/fixation.csv")

In [51]:
# check if there is nan in the list, if there is, raise error
for lis in fix_df['list_dur']:
    if np.isnan(lis).any():
        print("there is nan in the list")

In [54]:
for i in range(len(freq_df)):
    if np.isnan(freq_df['freq'][i]).any():
        print("there is nan in the list", i)

In [75]:
print(freq_df['freq'][40])

[0.33333333 0.33333333 0.33333333]


In [58]:
fix_df['list_dur'][34]

[0.09211129788395717,
 0.17188242751090696,
 0.21461349644500557,
 0.31156778937684965,
 0.2098249887832807]

In [59]:
scipy.stats.spearmanr(freq_df['freq'][34], fix_df['list_dur'][34])

/Users/first_principle/opt/anaconda3/envs/scan/lib/python3.9/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


SignificanceResult(statistic=nan, pvalue=nan)

In [81]:
sst_df['text'][40].split()

['reassuring,', 'retro', 'uplifter.']

In [3]:
import pandas as pd

task = "sst"
fix_df = pd.read_csv(f"data/{task}/fixation.csv", sep=',')
freq_df = pd.read_csv(f"data/{task}/freq.csv", sep=',')
# turn the list of string to list of list
fix_df['list_dur'] = fix_df['list_dur'].apply(ast.literal_eval)
freq_df['freq'] = freq_df['freq'].apply(ast.literal_eval)

In [41]:
import pandas as pd
import ast

task = "sst"
model = "bert_large"
tuned = "random"
fix_df = pd.read_csv(f"data/{task}/fixation.csv", sep=',')
freq_df = pd.read_csv(f"data/{task}/{model}_{tuned}_saliency.csv", sep=',')

fix_df['list_dur'] = fix_df['list_dur'].apply(ast.literal_eval)
freq_df['l1_grad'] = freq_df['l1_grad'].apply(ast.literal_eval)

In [42]:
import scipy.stats    
import numpy as np

df = pd.merge(fix_df, freq_df, on='sid')

spearman_l1 = []

for i in range(len(df)):
    if len(df['list_dur'][i]) == len(df['l1_grad'][i]):
        spearman_l1.append(scipy.stats.spearmanr(df['l1_grad'][i], df['list_dur'][i])[0])
    else:
        print("the length of the list is not the same")

print(np.mean(spearman_l1))


0.22984451193893649


In [4]:
import scipy.stats    
import numpy as np
import ast

df = pd.merge(freq_df, fix_df, on='sid')
spearman_l1 = []
# make list_dur and freq to list

    
for i in range(len(df)):
    if len(df['freq'][i]) != len(df['list_dur'][i]):
        print("sentence {} has different length of freq and list_dur".format(df['sid'][i]))
        continue
    spearman_l1.append(scipy.stats.spearmanr(df['freq'][i], df['list_dur'][i])[0])

# remove nan in spearman_l1
spearman_l1 = [x for x in spearman_l1 if str(x) != 'nan']
print(np.mean(spearman_l1))
print(spearman_l1)
print(len(spearman_l1))

    

0.6485531435370054
[0.6877651524512388, 0.7452777835998551, 0.5033244599486209, 0.8207826816681234, 0.7517124159880733, 0.7431209785167866, 0.7496841818306434, 0.6605782590758164, 0.5205022960939637, 0.8513733552879053, 0.379871264409569, 0.8829187134416479, 0.7387687194103584, 0.6627063412298289, 0.8263621207201486, 0.4678382377969518, 0.6443249279897771, 0.5464399518126823, 0.7926926584097701, 0.5146185029660212, 0.3333563164299625, 0.777437301245559, 0.7733545504844775, 0.7714285714285715, 0.8571428571428573, 0.42105522349350233, 0.7619047619047621, 0.6400431019968835, 0.8954053273686235, 0.2601529217201734, 0.7323529411764707, 0.8396728458480681, 0.7773158616991582, 0.6094316999565111, 0.7826237921249264, 0.8178995960439669, 0.7860082507317264, 0.7431505414602932, 0.7014035027756877, 0.379841990335979, 0.8827348295047495, 0.6, 0.7714285714285715, 0.36446563806256416, 0.39487598766475923, 0.5931426319344967, 0.642857142857143, 0.7320501594135719, 0.8306277919354929, 0.42857142857142

/Users/first_principle/opt/anaconda3/envs/scan/lib/python3.9/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


In [89]:
for i in range(len(spearman_l1)):
    if np.isnan(spearman_l1[i]):
        print("there is nan in the list", i)

there is nan in the list 40


# test


In [8]:
from analysis import *
df = read_data("pretrained", "sst")
corr_df = get_corr(df)

In [9]:
corr_df['bert_large'].mean()

0.5547754905049734